In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Vadim\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!



    Error: Couldn't link model to 'en'
    Creating a symlink in spacy/data failed. Make sure you have the required
    permissions and try re-running the command as admin, or use a
    virtualenv. You can still import the model as a module and call its
    load() method, or create the symlink manually.

    C:\Users\Vadim\Anaconda3\lib\site-packages\en_core_web_sm -->
    C:\Users\Vadim\Anaconda3\lib\site-packages\spacy\data\en


    Creating a shortcut link for 'en' didn't work (maybe you don't have
    admin permissions?), but you can still load the model via its full
    package name: nlp = spacy.load('{name}')
    Download successful but linking failed



Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

**Note**: Since processing all the text takes around ~5-10 minutes, in the cell below we are taking only the first tenth of each text. If you want to experiment, feel free to change the following code in the next cell:

```python
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
```
to 

```python
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
```

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
# Parse the cleaned novels. This can take a bit.
#nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [5]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [6]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    '''
    
    '''
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    '''
    returns dataframe with columns common_words + the sentense itself and the source document and 
     rows  is the sentence number 
    '''
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [7]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,different,unconnected,willingly,material,will,gratitude,modesty,captain,preside,thin,...,finger,bath,vain,opinion,grant,thump,crofts,sake,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [8]:
word_counts.tail()

,different,unconnected,willingly,material,will,gratitude,modesty,captain,preside,thin,...,finger,bath,vain,opinion,grant,thump,crofts,sake,text_sentence,text_source
412,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(But, it, was, not, a, merely, selfish, cautio...",Austen
413,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Had, she, not, imagined, herself, consulting,...",Austen
414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,"(The, belief, of, being, prudent, ,, and, self...",Austen
415,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(He, had, left, the, country, in, consequence, .)",Austen
416,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(A, few, months, had, seen, the, beginning, an...",Austen


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [9]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
# we will try to predict text source using freqency info 
#
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

C:\Users\Vadim\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.992

Test set score: 0.820359281437


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [10]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\Users\Vadim\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(250, 1647) (250,)
Training set score: 0.976

Test set score: 0.88622754491


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [11]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.964

Test set score: 0.814371257485


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data. Remember that for computation time concerns, we only took the first tenth of the Alice text. Emma is pretty long. **So in order to get comparable length texts, we take the first sixtieth of Emma**. Again, if you want to experiment, you can take the whole texts of each.

In [12]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:150])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to unite some of the best blessings of existence; an


In [13]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [14]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [15]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 50
Processing row 100
done


In [16]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.633027522936


col_0,Austen,Carroll
row_0,,
Austen,120,20
Carroll,60,18


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance. 
Suggested avenues of investigation include: 
* Other modeling techniques (SVM?),
* making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), 
* making sentence-level features (number of words, amount of punctuation), or 
* including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  

Make sure to: 
 * design your models on the test set, or 
 * use cross_validation with multiple folds, and 
 * see if you can get **accuracy above 90%**.  



In [17]:
# SVM
from sklearn.svm import LinearSVC
model_svc = LinearSVC(random_state=42, tol=1e-5) # Linear Kernel
#Train the model using the training sets
model_svc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model_svc.predict(X_test)
# evaluation 
print('SVM linear kernel Training set score:', model_svc.score(X_train, y_train))
print('\nSVM linear kernel Test set score:', model_svc.score(X_test, y_test))

SVM linear kernel Training set score: 0.996

SVM linear kernel Test set score: 0.880239520958


In [18]:
from sklearn.model_selection import cross_val_score
model_svc1 = LinearSVC(random_state=42, tol=1e-3) # Linear Kernel
model_svc1.fit(X_train, y_train)
scores = cross_val_score(model_svc1, X_train, y_train, cv=5)
print(scores)
y_pred = model_svc1.predict(X_test)
print('SVM linear kernel Training set score:', model_svc1.score(X_train, y_train))
print('\nSVM linear kernel Test set score:', model_svc1.score(X_test, y_test))

[ 0.82352941  0.96078431  0.9         0.85714286  0.85714286]
SVM linear kernel Training set score: 0.996

SVM linear kernel Test set score: 0.880239520958


*Logistic regression*  results :
    * Training set score: 0.976
    * Test set score: 0.88622754491
--------------------------------------------    
*SVC with linear kernel* results: 
    * Train 0.996
    * Test  0.880

----------------------------
What could be done to improve it?
 * try ensemble approach 
 * try gridsearch 
 
 * use entire standard spacy pipeline 
 
 


In [19]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Training score: ",gnb.score(X_train,y_train))
print("Testing score:",gnb.score(X_test,y_test))

print(pd.crosstab(y_test,gnb_pred))


Training score:  0.98
Testing score: 0.910179640719
col_0        Austen  Carroll
text_source                 
Austen          108        6
Carroll           9       44


Naive Bayes classifiers tend to perform especially well in one of the following situations:

When the naive assumptions actually match the data (very rare in practice)
For very well-separated categories, when model complexity is less important
For very high-dimensional data, when model complexity is less important

**Training score:  0.98
Testing score: 0.910179640719**


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

In [28]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [63]:
def text_prep(text_name):
    text_loaded = gutenberg.raw(text_name)
    # The Chapter indicator is idiosyncratic
    text_re = re.sub(r'Chapter \d+', '', text_loaded)
    clean_text=text_cleaner(text_re[:int(len(text_re)/10)]) # load text
    tdoc = nlp(clean_text)  # make object
    print('len of ' + text_name )
    print(len(tdoc))
    return tdoc

def df_prep(text1, label1, text2, label2):
    # use prepared nlp doc to make final X,Y
    t1doc =  text_prep(text1)
    t1_sents = [[sent, label1] for sent in t1doc.sents] # 
    print('text 1 ') 
    print(len(t1_sents))
    t2doc =  text_prep(text2)
    t2_sents = [[sent, label2] for sent in t2doc.sents] # 
    print('text 2 ') 
    print(len(t2_sents))
    # Combine the sentences from the two novels into one data frame.
    sentences = pd.DataFrame(t1_sents + t2_sents )
    
    t1_words = bag_of_words(t1doc)
    t2_words = bag_of_words(t2doc)
    common_words = set(t2_words + t1_words)
    return bow_features(sentences, common_words)



In [61]:
def compare_texts(text1, label1,cut1, text2, label2, cut2):
    word_counts = df_prep(text1, label1, text2, label2)
    Y = word_counts['text_source']
    X = np.array(word_counts.drop(['text_sentence','text_source'], 1))
    
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=42)
    print(X_train.shape)
    gnb = GaussianNB()
    gnb_pred = gnb.fit(X_train, y_train).predict(X_test)
    print("Training score: ",gnb.score(X_train,y_train))
    print("Testing score:",gnb.score(X_test,y_test))
    print(pd.crosstab(y_test,gnb_pred))
    

In [64]:
compare_texts('carroll-alice.txt','Alice','chesterton-ball.txt','C-Ball')
compare_texts('carroll-alice.txt','Alice','whitman-leaves.txt','W-leaves')

len of carroll-alice.txt
3352
text 1 
131
len of chesterton-ball.txt
9558
text 2 
444
Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
(460, 1832)
Training score:  0.976086956522
Testing score: 0.808695652174
col_0        Alice  C-Ball
text_source               
Alice           18      10
C-Ball          12      75
len of carroll-alice.txt
3352
text 1 
131
len of whitman-leaves.txt
15087
text 2 
612
Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600
Processing row 650
Processing row 700
(594, 2229)
Training score:  0.93265993266
Testing score: 0.865771812081
col_0        Alice  W-leaves
text_source                 
Alice   

In [65]:
compare_texts('austen-persuasion.txt','austen-persuasion','bryant-stories.txt','bryant-stories')
compare_texts('austen-persuasion.txt','austen-persuasion','blake-poems.txt','blake-poems')

len of austen-persuasion.txt
9631
text 1 
286
len of bryant-stories.txt
5864
text 2 
314
Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
(480, 1749)
Training score:  0.983333333333
Testing score: 0.966666666667
col_0              austen-persuasion  bryant-stories
text_source                                         
austen-persuasion                 58               3
bryant-stories                     1              58
len of austen-persuasion.txt
9631
text 1 
286
len of blake-poems.txt
842
text 2 
48
Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
(267, 1503)
Training score:  0.98127340824
Testing score: 0.910447761194
col_0              austen-persuasion  blake-poems
text_source                                      
austen-persua